##  A Golden Path: Context
Hopefully the information and techniques provided here lessen your time and effort to obtain a working configuration.  

Note: This Jupyter Notebook was constructed with Microsoft Visual Code.  To jump to a section of interest used the OUTLINE view within the Explorer option in the Activity Bar. 

## 01  Hardware and software platforms

### 01.01  general development machine, Apple Silicon

In [ ]:
%%bash
printf -- '  %s  --  developer hardware \n' "$(sysctl -n machdep.cpu.brand_string | cut -c 1-8)"
printf -- '%s %s  --  developer operating system' "$(sw_vers --productName)" "$(sw_vers --productVersion)"

  Apple M4  --  developer hardware 
macOS 15.6  --  developer operating system

### 01.02  COBOL development machine, Apple MacBook Pro with Intel.
*  IBM COBOL for Linux x86 pre-reqs one of three operating systems.  RHEL 9 is my choice because of previous experience.

Note:  My QNAP NAS with an Intel® Celeron® Processor N5095 lacks the [documented requirement for the avx2](https://www.ibm.com/docs/en/cobol-linux-x86/1.2.0?topic=prerequisites-system) instruction set.

In [1]:
%%bash
# Red Hat command to provide operating system summary.

strng=$'ssh -T ic4lx86 \'hostnamectl\''
printf '\n>>>%s<<<\n\n' "$strng"
eval $strng


>>>ssh -T ic4lx86 'hostnamectl'<<<

 Static hostname: ic4lx86
       Icon name: computer-vm
         Chassis: vm 🖴
      Machine ID: 3bb72366d1e44ce883da69489ea40b69
         Boot ID: 64830cc96116489684b3d888b6a58def
  Virtualization: vmware
Operating System: Red Hat Enterprise Linux 9.6 (Plow)
     CPE OS Name: cpe:/o:redhat:enterprise_linux:9::baseos
          Kernel: Linux 5.14.0-570.28.1.el9_6.x86_64
    Architecture: x86-64
 Hardware Vendor: VMware, Inc.
  Hardware Model: VMware20,1
Firmware Version: VMW201.00V.24006586.B64.2406042154


In [ ]:
%%bash
# Ask the operating system about the hardware platform.
## Need an Intel processor with support for the avx2 instruction set.

strng='ssh -T ic4lx86 '
strng+=$'\'grep --max-count 1 vendor_id /proc/cpuinfo && '
strng+=$'grep --only-matching --max-count 1 \'avx2\' /proc/cpuinfo \''
printf '\n>>>%s<<<\n\n' "$strng"
eval $strng


>>>ssh -T ic4lx86 'grep --max-count 1 vendor_id /proc/cpuinfo && grep --only-matching --max-count 1 'avx2' /proc/cpuinfo '<<<

vendor_id	: GenuineIntel
avx2


#### Linux instance for COBOL, configure bash for virtuser
Red Hat Developer blog, Rob Terzi, 2018-08-15  --  How to enable sudo on Red Hat Enterprise Linux
https://developers.redhat.com/blog/2018/08/15/how-to-enable-sudo-on-rhel#tl_dr__basic_sudo  


```
set -o vi

# 2025-03-04T set environment variables for Gnu ls
export BLOCK_SIZE="'1"
export TIME_STYLE='+%Y-%m-%d_%H:%M:%S_%z'

## Remember to escape $PWD to prevent expansion at first execution.
export PS1="\n[grace \$(date --iso-8601=seconds )  \$PWD]\n\$ "

# IBM COBOL for Linux on x86 values
export PATH=$PATH:/opt/ibm/cobol/1.2.0/bin:/opt/ibm/cobol/1.2.0/rte/bin:/opt/ibm/cobol/1.2.0/debug/usr/bin

# 2025-07-28 "IBM COBOL for Linux on x86 1.2: Installation Guide" p015
if [[ "$MANPATH" ]]; then
  export MANPATH=$MANPATH:/opt/ibm/cobol/1.2.0/man:/opt/ibm/cobol/rte/man
else
  export MANPATH=/opt/ibm/cobol/1.2.0/man:/opt/ibm/cobol/rte/man
fi
```

Example of setting sudoers file.

```
[01.rhel9.nas 2025-08-18T20:54:59-05:00  /home/virtuser]
$ su --login root
Password:
[root@01 ~]# echo -e "virtuser\tALL=(ALL)\tNOPASSWD: ALL" > /etc/sudoers.d/020_virtuser && cat /etc/sudoers.d/020_virtuser
virtuser	ALL=(ALL)	NOPASSWD: ALL
[root@01 ~]# exit
logout

[01.rhel9.nas 2025-08-18T20:58:30-05:00  /home/virtuser]
$ sudo ls -lt /etc/sudoers.d/
total 4
-rw-r--r--. 1 root root 33 Aug 18 20:58 020_virtuser
```

#### Review Gnu Complier Collection levels
See Documentation IBM COBOL for Linux on x86 [Software requirements: required programs](https://www.ibm.com/docs/en/cobol-linux-x86/1.2.0?topic=prerequisites-system#sysprereqs__required__title__1◊)

In [ ]:
%%bash
# virtuser is now a member of sudoers.  Review Gnu Complier Collection packages.
## Note, this command may take five seconds, or so.

strng=$'ssh ic4lx86 -T \'sudo dnf list installed '
strng+='gcc '
strng+='glibc '
strng+='glibc-devel '
strng+='libgcc '
strng+='libstdc++ '
strng+=$'| column | tail -n +3 | nl\''

printf '\n>>>%s<<<\n\n' "$strng"
eval $strng


>>>ssh ic4lx86 -T 'sudo dnf list installed gcc glibc glibc-devel libgcc libstdc++ | column | tail -n +3 | nl'<<<
     1	gcc.x86_64              11.5.0-5.el9_5         @rhel-9-for-x86_64-appstream-rpms
     2	glibc.x86_64            2.34-168.el9_6.23      @rhel-9-for-x86_64-baseos-rpms   
     3	glibc-devel.x86_64      2.34-168.el9_6.23      @rhel-9-for-x86_64-appstream-rpms
     4	libgcc.x86_64           11.5.0-5.el9_5         @rhel-9-for-x86_64-baseos-rpms   
     5	libstdc++.x86_64        11.5.0-5.el9_5         @rhel-9-for-x86_64-baseos-rpms   


## 02  PostgreSQL

###  02.01  Postgres.app on developer machine -- the server
I used the version of PostgreSQL current in Summer 2025.  Postgres.app automates the installation and dayone tasks.  Highly recommended.  To Complement Postgress.app I used pgAdmin, current release, for the database administartion and observation. 

### 02.02 PostgreSQL installed on Linux for IBM COBOL for Linux on x86
The IBM Documentation [Compiling and linking a COBOL program with embedded SQL statements in PostgreSQL](https://www.ibm.com/docs/en/cobol-linux-x86/1.2.0?topic=ppe-compiling-linking-cobol-program-embedded-sql-statements-in-postgresql) describes the need for the PostgreSQL ECPG library.  
"[The libecpg library primarily contains “hidden” functions that are used to implement the functionality expressed by the embedded SQL commands.](https://www.postgresql.org/docs/current/ecpg-library.html)
ECPG, the embedded SQL preprocessor for C programs, is  included as part of the PostgreSQL development packages.  It is not part in a RHEL provided repository.

In [63]:
%%bash
# RHEL does not provided the PostgreSQL developer libraries.
## This command may take several seconds to complete.

strng=$'ssh ic4lx86 -T \'sudo dnf list available postgresql*\''
printf '\n>>>%s<<<\n\n' "$strng"
eval $strng


>>>ssh ic4lx86 -T 'sudo dnf list available postgresql*'<<<

Updating Subscription Management repositories.
Red Hat Enterprise Linux 9 for x86_64 - BaseOS   18 kB/s | 4.1 kB     00:00    
Red Hat Enterprise Linux 9 for x86_64 - BaseOS   25 MB/s |  72 MB     00:02    
Red Hat Enterprise Linux 9 for x86_64 - AppStre  25 kB/s | 4.5 kB     00:00    
Red Hat Enterprise Linux 9 for x86_64 - AppStre  19 MB/s |  66 MB     00:03    
Last metadata expiration check: 0:00:13 ago on Thu 21 Aug 2025 11:33:19 AM CDT.
Available Packages
postgresql.x86_64              13.20-1.el9_5    rhel-9-for-x86_64-appstream-rpms
postgresql-contrib.x86_64      13.20-1.el9_5    rhel-9-for-x86_64-appstream-rpms
postgresql-jdbc.noarch         42.2.28-1.el9_3  rhel-9-for-x86_64-appstream-rpms
postgresql-odbc.x86_64         12.02.0000-6.el9 rhel-9-for-x86_64-appstream-rpms
postgresql-plperl.x86_64       13.20-1.el9_5    rhel-9-for-x86_64-appstream-rpms
postgresql-plpython3.x86_64    13.20-1.el9_5    rhel-9-for-x86_64-ap

#### 02.02.01 PostgresSQL instructions for installing product binaries and development libraries on Linux (RHEL)

At [Linux downloads (Red Hat family)](https://www.postgresql.org/download/linux/redhat/) there is prompter to describe what is needed from PostgreSQL:
*  Select version: 17
*  Select platform: Red Hat Enterprise, Rocky, AlmaLinux or Oracale version 9
*  Select architecture: x86_64 

```
# Install the repository RPM:
sudo dnf install -y https://download.postgresql.org/pub/repos/yum/reporpms/EL-9-x86_64/pgdg-redhat-repo-latest.noarch.rpm
# Disable the built-in PostgreSQL module:
sudo dnf -qy module disable postgresql
# Install PostgreSQL:
sudo dnf install -y postgresql17-server
```

In [64]:
%%bash
# From postgresql.org install the repository RPM.
## This command may take twenty seconds to complete.


strng=$'ssh ic4lx86 -T \'sudo dnf install --assumeyes '
strng+=$'https://download.postgresql.org/pub/repos/yum/reporpms/EL-9-x86_64/pgdg-redhat-repo-latest.noarch.rpm\''
printf '\n>>>%s<<<\n\n' "$strng"
eval $strng


>>>ssh ic4lx86 -T 'sudo dnf install --assumeyes https://download.postgresql.org/pub/repos/yum/reporpms/EL-9-x86_64/pgdg-redhat-repo-latest.noarch.rpm'<<<

Updating Subscription Management repositories.
Last metadata expiration check: 0:00:32 ago on Thu 21 Aug 2025 11:33:19 AM CDT.
pgdg-redhat-repo-latest.noarch.rpm               41 kB/s |  12 kB     00:00    
Dependencies resolved.
 Package                Architecture Version           Repository          Size
Installing:
 pgdg-redhat-repo       noarch       42.0-54PGDG       @commandline        12 k

Transaction Summary
Install  1 Package

Total size: 12 k
Installed size: 17 k
Running transaction check
Transaction check succeeded.
Running transaction test
Transaction test succeeded.
Running transaction
  Preparing        :                                                        1/1 
  Installing       : pgdg-redhat-repo-42.0-54PGDG.noarch                    1/1 
  Verifying        : pgdg-redhat-repo-42.0-54PGDG.noarch                 

In [65]:
%%bash
# Disable the built-in PostgreSQL module.
## This command may take twenty seconds to complete.

strng=$'ssh ic4lx86 -T \'sudo dnf module disable postgresql --quiet --assumeyes\''
printf '\n>>>%s<<<\n\n' "$strng"
eval $strng


>>>ssh ic4lx86 -T 'sudo dnf module disable postgresql --quiet --assumeyes'<<<



Importing GPG key 0x08B40D20:
 Userid     : "PostgreSQL RPM Repository <pgsql-pkg-yum@lists.postgresql.org>"
 Fingerprint: D4BF 08AE 67A0 B4C7 A1DB CCD2 40BC A2B4 08B4 0D20
 From       : /etc/pki/rpm-gpg/PGDG-RPM-GPG-KEY-RHEL
Importing GPG key 0x08B40D20:
 Userid     : "PostgreSQL RPM Repository <pgsql-pkg-yum@lists.postgresql.org>"
 Fingerprint: D4BF 08AE 67A0 B4C7 A1DB CCD2 40BC A2B4 08B4 0D20
 From       : /etc/pki/rpm-gpg/PGDG-RPM-GPG-KEY-RHEL
Importing GPG key 0x08B40D20:
 Userid     : "PostgreSQL RPM Repository <pgsql-pkg-yum@lists.postgresql.org>"
 Fingerprint: D4BF 08AE 67A0 B4C7 A1DB CCD2 40BC A2B4 08B4 0D20
 From       : /etc/pki/rpm-gpg/PGDG-RPM-GPG-KEY-RHEL
Importing GPG key 0x08B40D20:
 Userid     : "PostgreSQL RPM Repository <pgsql-pkg-yum@lists.postgresql.org>"
 Fingerprint: D4BF 08AE 67A0 B4C7 A1DB CCD2 40BC A2B4 08B4 0D20
 From       : /etc/pki/rpm-gpg/PGDG-RPM-GPG-KEY-RHEL
Importing GPG key 0x08B40D20:
 Userid     : "PostgreSQL RPM Repository <pgsql-pkg-yum@lists.post

In [66]:
%%bash
# Validate that the two packages of interest are available.
## This command may take ten seconds to complete.

strng=$'ssh ic4lx86 -T \'sudo dnf info postgresql17 postgresql17-devel\''
printf '\n>>>%s<<<\n\n' "$strng"
eval $strng


>>>ssh ic4lx86 -T 'sudo dnf info postgresql17 postgresql17-devel'<<<

Updating Subscription Management repositories.
Last metadata expiration check: 0:00:12 ago on Thu 21 Aug 2025 11:34:16 AM CDT.
Available Packages
Name         : postgresql17
Version      : 17.6
Release      : 1PGDG.rhel9
Architecture : x86_64
Size         : 1.9 M
Source       : postgresql17-17.6-1PGDG.rhel9.src.rpm
Repository   : pgdg17
Summary      : PostgreSQL client programs and libraries
URL          : https://www.postgresql.org/
License      : PostgreSQL
Description  : PostgreSQL is an advanced Object-Relational database management system (DBMS).
             : The base postgresql package contains the client programs that you'll need to
             : access a PostgreSQL DBMS server. These client programs can be located on the
             : same machine as the PostgreSQL server, or on a remote machine that accesses a
             : PostgreSQL server over a network connection. The PostgreSQL server can be found

#### 02.02.02  postgresql17 install product binaries

In [67]:
%%bash
# Install postgresql17.
## This command may take twenty seconds to complete.

strng=$'ssh ic4lx86 -T \'sudo dnf install postgresql17 --quiet --assumeyes  &&  '
strng+=$'psql --version \''
printf '\n>>>%s<<<\n\n' "$strng"
eval $strng


>>>ssh ic4lx86 -T 'sudo dnf install postgresql17 --quiet --assumeyes  &&  psql --version '<<<



Importing GPG key 0x08B40D20:
 Userid     : "PostgreSQL RPM Repository <pgsql-pkg-yum@lists.postgresql.org>"
 Fingerprint: D4BF 08AE 67A0 B4C7 A1DB CCD2 40BC A2B4 08B4 0D20
 From       : /etc/pki/rpm-gpg/PGDG-RPM-GPG-KEY-RHEL



Installed:
  postgresql17-17.6-1PGDG.rhel9.x86_64                                          
  postgresql17-libs-17.6-1PGDG.rhel9.x86_64                                     

psql (PostgreSQL) 17.6


#### 02.02.03  PostgresSQL development library and perl(IPC::Run) pre-req

The first time attempt with  
```
sudo dnf install postgresql17-devel \--quiet \--assumeyes
```

resulted in   
```
Error:   
 Problem: cannot install the best candidate for the job  
  \- nothing provides perl(IPC::Run) needed by postgresql17-devel-17.6-1PGDG.rhel9.x86\_64 from pgdg17
```

Investigating with Google Search I learned about Red Hat’s [CodeReady Linux Builder repository.](https://access.redhat.com/solutions/4180391)  This is where Red Hat provides access to repositories they donot support.  The Red Hat Customer Portal Knowledgebase article “[Where are perl-IPC-Run and perl-IO-Tty packages in RHEL 8 and RHEL 9](https://access.redhat.com/solutions/7017597)” provides example commands for obtaining the perl-IPC-run package and a link for understanding CodeReady Linux Builder repository.

* \# subscription-manager repos \--enable codeready-builder-for-rhel-9-x86\_64-rpms  
* \# yum install perl-IPC-Run


In [68]:
%%bash
# Configure the RHEL 9 instance to access Red Hat's CodeReady Linux Builder repository.
## This command may take thirty seconds to complete.

strng=$'ssh ic4lx86 -T \'sudo subscription-manager repos --enable codeready-builder-for-rhel-9-x86_64-rpms\''
printf '\n>>>%s<<<\n\n' "$strng"
eval $strng


>>>ssh ic4lx86 -T 'sudo subscription-manager repos --enable codeready-builder-for-rhel-9-x86_64-rpms'<<<

Repository 'codeready-builder-for-rhel-9-x86_64-rpms' is enabled for this system.


In [69]:
%%bash
# Install perl-IPC-Run
## This command may take twenty seconds to complete.

strng=$'ssh ic4lx86 -T \'sudo dnf install perl-IPC-Run --quiet --assumeyes\''
printf '\n>>>%s<<<\n\n' "$strng"
eval $strng


>>>ssh ic4lx86 -T 'sudo dnf install perl-IPC-Run --quiet --assumeyes'<<<



Importing GPG key 0xFD431D51:
 Userid     : "Red Hat, Inc. (release key 2) <security@redhat.com>"
 Fingerprint: 567E 347A D004 4ADE 55BA 8A5F 199E 2F91 FD43 1D51
 From       : /etc/pki/rpm-gpg/RPM-GPG-KEY-redhat-release
Importing GPG key 0x5A6340B3:
 Userid     : "Red Hat, Inc. (auxiliary key 3) <security@redhat.com>"
 Fingerprint: 7E46 2425 8C40 6535 D56D 6F13 5054 E4A4 5A63 40B3
 From       : /etc/pki/rpm-gpg/RPM-GPG-KEY-redhat-release



Installed:
  perl-IO-Tty-1.16-4.el9.x86_64       perl-IPC-Run-20200505.0-6.el9.noarch      



In [ ]:
%%bash
# Install postgresql17-devel.
## This command may take up to two minutes to complete.

strng=$'ssh ic4lx86 -T \'sudo dnf install postgresql17-devel --quiet --assumeyes  &&  '
strng+=$'find /usr/pgsql-17/ -type f -name libecpg.a  -exec ls -l {} \+\''
printf '\n>>>%s<<<\n\n' "$strng"
eval $strng


>>>ssh ic4lx86 -T 'sudo dnf install postgresql17-devel --quiet --assumeyes  &&  find /usr/pgsql-17/ -type f -name libecpg.a  -exec ls -l {} \+'<<<


Installed:
  clang-19.1.7-2.el9.x86_64                                                     
  clang-devel-19.1.7-2.el9.x86_64                                               
  clang-libs-19.1.7-2.el9.x86_64                                                
  clang-resource-filesystem-19.1.7-2.el9.x86_64                                 
  clang-tools-extra-19.1.7-2.el9.x86_64                                         
  cmake-filesystem-3.26.5-2.el9.x86_64                                          
  compiler-rt-19.1.7-2.el9.x86_64                                               
  environment-modules-5.3.0-1.el9.x86_64                                        
  gcc-toolset-14-binutils-2.41-4.el9_6.1.x86_64                                 
  gcc-toolset-14-gcc-14.2.1-7.1.el9.x86_64                                      
  gcc-toolset-14-gcc-c++-14.2

### 02.03  Setup database AD22 and access with user michael.

In [5]:
%%bash
# Implement what Anthony DeBarros describes in his book 
## "Practical SQL, A Beginner’s Guide to Storytelling with Data", 2022.
../SCRIPTS/ad22_ch02_lst01.sh


***  2025-08-21T20:23:37:-0500 --  Listing ad22-02-01 CREATE DATABASE analysis;  ***
------------------------------------------------------------------------------------
CREATE DATABASE

---  2025-08-21T20:23:37:-0500 --  echo $?  --  zero means "All is well."  ---
------------------------------------------------------------------------------
0

---  2025-08-21T20:23:37:-0500 --  CREATE the table if does not exist. Then INSERT data.  ---
---------------------------------------------------------------------------------------------
CREATE TABLE IF NOT EXISTS teachers
(
    id bigserial,
    first_name varchar(25),
    last_name varchar(50),
    school varchar(50),
    hire_date date,
    salary numeric
);
CREATE TABLE
INSERT INTO teachers (first_name, last_name, school, hire_date, salary)
VALUES ('Janet', 'Smith', 'F.D. Roosevelt HS', '2011-10-30', 36200),
       ('Lee', 'Reynolds', 'F.D. Roosevelt HS', '1993-05-22', 65000),
       ('Samuel', 'Cole', 'Myers Middle School', '2005-08-01',

In [6]:
%%bash
# Add a role to login and access the teachers table.
psql --dbname AD22 --echo-all --echo-queries <<EoF
CREATE ROLE michael LOGIN PASSWORD 'Stonebraker';
GRANT ALL PRIVILEGES ON teachers TO michael;
EoF

strng=$'psql --dbname AD22 --username michael --command \'TABLE teachers;\''
printf '\n>>>%s<<<\n\n' "$strng"
eval $strng

CREATE ROLE michael LOGIN PASSWORD 'Stonebraker';


ERROR:  role "michael" already exists


GRANT ALL PRIVILEGES ON teachers TO michael;
GRANT

>>>psql --dbname AD22 --username michael --command 'TABLE teachers;'<<<

 id | first_name | last_name |       school        | hire_date  | salary 
----+------------+-----------+---------------------+------------+--------
  1 | Janet      | Smith     | F.D. Roosevelt HS   | 2011-10-30 |  36200
  2 | Lee        | Reynolds  | F.D. Roosevelt HS   | 1993-05-22 |  65000
  3 | Samuel     | Cole      | Myers Middle School | 2005-08-01 |  43500
  4 | Samantha   | Bush      | Myers Middle School | 2011-10-30 |  36200
  5 | Betty      | Diaz      | Myers Middle School | 2005-08-30 |  43500
  6 | Kathleen   | Roush     | F.D. Roosevelt HS   | 2010-10-22 |  38500
(6 rows)



### 02.04  Access AD22 from the remote COBOL machine

In [38]:
%%bash
# Find IPv4 address for this machine, PostgreSQL server, and configure its PostgreSQL listen address.

strng=$'printf \'%s -- the value for listen_addresses in postgresql.config\n\' '
strng+=$'"$(ifconfig en0 inet | tail -n 1 | cut -d \' \' -f2)"'
printf '\n>>>%s<<<\n\n' "$strng"
eval $strng

strng=$'sed -n \'844,845p\' /Users/mbp9/Library/Application\ Support/Postgres/var-17/postgresql.conf | nl -v 844'
printf '\n>>>%s<<<\n\n' "$strng"
eval $strng


>>>printf '%s -- the value for listen_addresses in postgresql.config
' "$(ifconfig en0 inet | tail -n 1 | cut -d ' ' -f2)"<<<

192.168.5.23 -- the value for listen_addresses in postgresql.config 
>>>sed -n '844,845p' /Users/mbp9/Library/Application\ Support/Postgres/var-17/postgresql.conf | nl -v 844<<<

   844	# 2025-07-28M  mbp9 listens on this network interface for connections.
   845	listen_addresses = '192.168.5.23'


In [39]:
%%bash
# Set host-based authentication options. 
strng=$'sed -n \'129,134p\' /Users/mbp9/Library/Application\ Support/Postgres/var-17/pg_hba.conf | nl -v 129'
printf '\n>>>%s<<<\n\n' "$strng"
eval $strng



>>>sed -n '129,134p' /Users/mbp9/Library/Application\ Support/Postgres/var-17/pg_hba.conf | nl -v 129<<<

   129	# 2025-07-28M Configure connections.
   130	## subject to MD5 authentication.
   131	# TYPE  DATABASE        USER            ADDRESS                 METHOD
   132	# host  	all             all             samenet         	trust
   133	host  	all             all             192.168.5.23/32         trust
   134	host    AD22            michael         192.168.5.88/32      	scram-sha-256


In [40]:
%%bash
# Review PostgreSQL environment variables for access the PostgreSQL server, then test.
strng='ssh ic4lx86 -T '
strng+=$'\'nl pgsql_env_vars && '
strng+=$'source pgsql_env_vars && echo &&'
strng+=$'psql --command "SELECT version();"\''
printf '\n>>>%s<<<\n\n' "$strng"
eval $strng


>>>ssh ic4lx86 -T 'nl pgsql_env_vars && source pgsql_env_vars && echo &&psql --command "SELECT version();"'<<<

     1	export PGDATABASE=AD22
     2	export PGHOST=192.168.5.23
     3	export PGPASSWORD=Stonebraker
     4	export PGPORT=5432
     5	export PGUSER=michael

                                                             version                                                             
---------------------------------------------------------------------------------------------------------------------------------
 PostgreSQL 17.6 (Postgres.app) on aarch64-apple-darwin23.6.0, compiled by Apple clang version 15.0.0 (clang-1500.3.9.4), 64-bit
(1 row)



In [41]:
%%bash
# Review PostgreSQL environment variables for access the PostgreSQL server, then test.
strng='ssh ic4lx86 -T '
strng+=$'\'nl pgsql_env_vars && '
strng+=$'source pgsql_env_vars && echo &&'
strng+=$'psql --command "TABLE teachers;"\''
printf '\n>>>%s<<<\n\n' "$strng"
eval $strng


>>>ssh ic4lx86 -T 'nl pgsql_env_vars && source pgsql_env_vars && echo &&psql --command "TABLE teachers;"'<<<

     1	export PGDATABASE=AD22
     2	export PGHOST=192.168.5.23
     3	export PGPASSWORD=Stonebraker
     4	export PGPORT=5432
     5	export PGUSER=michael

 id | first_name | last_name |       school        | hire_date  | salary 
----+------------+-----------+---------------------+------------+--------
  1 | Janet      | Smith     | F.D. Roosevelt HS   | 2011-10-30 |  36200
  2 | Lee        | Reynolds  | F.D. Roosevelt HS   | 1993-05-22 |  65000
  3 | Samuel     | Cole      | Myers Middle School | 2005-08-01 |  43500
  4 | Samantha   | Bush      | Myers Middle School | 2011-10-30 |  36200
  5 | Betty      | Diaz      | Myers Middle School | 2005-08-30 |  43500
  6 | Kathleen   | Roush     | F.D. Roosevelt HS   | 2010-10-22 |  38500
(6 rows)



## 03  Install IBM COBOL for Linux on X86
*  The evalulation version for IBM COBOL for Linux on x86 1.2 available Summer of 2025
*  Current maintance from IBM Fix Central, 1.2.0.7.  1.2.0.6 is the first IBM COBOL for Linux release to support PostgreSQL access
*  Broadcom VMWare Fusion file sharing for making the download IBM files available to the virtual machines.

### 03.01  Install the IBM product.

In [7]:
%%bash
# Confirm the IBM download is available

strng='ssh ic4lx86 -T '
strng+=$'\'ls -lt /mnt/hgfs/IBM-COBOL-1.2_EVAL/ && '
strng+='echo && '
strng+=$'ls -1 /mnt/hgfs/IBM-COBOL-1.2_EVAL/*gz\''
printf '\n>>>%s<<<\n\n' "$strng"
eval $strng


>>>ssh ic4lx86 -T 'ls -lt /mnt/hgfs/IBM-COBOL-1.2_EVAL/ && echo && ls -1 /mnt/hgfs/IBM-COBOL-1.2_EVAL/*gz'<<<

total 120432
-rw-r--r--. 1 502 games 123322169 Jul 26 14:09 IBM_COBOL_V1.2.0_LINUX_EVAL.x86-64.241122.tar.gz

/mnt/hgfs/IBM-COBOL-1.2_EVAL/IBM_COBOL_V1.2.0_LINUX_EVAL.x86-64.241122.tar.gz


In [ ]:
%%bash
# Make directory for .tar.gz expansion.  Expand the file, then check for contents.

strng='ssh ic4lx86 -T '
strng+=$'\'mkdir /home/virtuser/EXTRACT-COBOL && '
strng+='cd /home/virtuser/EXTRACT-COBOL && '
strng+='tar --extract --gunzip --file /mnt/hgfs/IBM-COBOL-1.2_EVAL/IBM_COBOL_V1.2.0_LINUX_EVAL.x86-64.241122.tar.gz && '
strng+=$'tree -d\''
printf '\n>>>%s<<<\n\n' "$strng"
eval $strng


>>>ssh ic4lx86 -T 'mkdir /home/virtuser/EXTRACT-COBOL && cd /home/virtuser/EXTRACT-COBOL && tar --extract --gunzip --file /mnt/hgfs/IBM-COBOL-1.2_EVAL/IBM_COBOL_V1.2.0_LINUX_EVAL.x86-64.241122.tar.gz && tree -d'<<<

.
└── images
    ├── rhel
    ├── sles
    └── ubuntu

4 directories


In [12]:
%%bash
# Install IBM COBOL for Linux on x86.
## This may take 30 seconds to complete.

strng='ssh ic4lx86 -T '
strng+=$'\'cd /home/virtuser/EXTRACT-COBOL && '
strng+=$'/home/virtuser/EXTRACT-COBOL/install\''
printf '\n>>>%s<<<\n\n' "$strng"
eval $strng


>>>ssh ic4lx86 -T 'cd /home/virtuser/EXTRACT-COBOL && /home/virtuser/EXTRACT-COBOL/install'<<<

/usr/bin/sudo rpm -Uvh /home/virtuser/EXTRACT-COBOL/images/rhel/*.rpm
Verifying...                          ########################################
Preparing...                          ########################################
Updating / installing...
cobol.cmp.license-eval.1.2.0-1.2.0.5-2########################################
cobol.rte.1.2.0-1.2.0.5-241115        ########################################
cobol.cmp.1.2.0-1.2.0.5-241115        ########################################
cobol.dbg.1.2.0-1.2.0.5-241115        ########################################
Installation and configuration successful


### 03.02  Install the IBM product's current maintenance, Summer 2025.

In [13]:
%%bash
# Confirm the IBM download is available

strng='ssh ic4lx86 -T '
strng+=$'\'ls -l /mnt/hgfs/FC_IBM-COBOL-1.2.0.7/IBM_COBOL_1.2.0.7_LINUX.x86-64.tar.gz\''
printf '\n>>>%s<<<\n\n' "$strng"
eval $strng


>>>ssh ic4lx86 -T 'ls -l /mnt/hgfs/FC_IBM-COBOL-1.2.0.7/IBM_COBOL_1.2.0.7_LINUX.x86-64.tar.gz'<<<

-rw-r--r--. 1 502 games 117461910 Jul 23 19:35 /mnt/hgfs/FC_IBM-COBOL-1.2.0.7/IBM_COBOL_1.2.0.7_LINUX.x86-64.tar.gz


In [14]:
%%bash
# Make directory for .tar.gz expansion.  Expand the file, then check for contents.

strng='ssh ic4lx86 -T '
strng+=$'\'mkdir /home/virtuser/EXTRACT-FC && '
strng+='cd /home/virtuser/EXTRACT-FC && '
strng+='tar --extract --gunzip --file /mnt/hgfs/FC_IBM-COBOL-1.2.0.7/IBM_COBOL_1.2.0.7_LINUX.x86-64.tar.gz && '
strng+=$'tree -d\''
printf '\n>>>%s<<<\n\n' "$strng"
eval $strng


>>>ssh ic4lx86 -T 'mkdir /home/virtuser/EXTRACT-FC && cd /home/virtuser/EXTRACT-FC && tar --extract --gunzip --file /mnt/hgfs/FC_IBM-COBOL-1.2.0.7/IBM_COBOL_1.2.0.7_LINUX.x86-64.tar.gz && tree -d'<<<

.
└── images
    ├── rhel
    ├── sles
    └── ubuntu

4 directories


In [15]:
%%bash
# Install IBM Fix Central maintenance for COBOL for Linux on x86.
## This may take 30 seconds to complete.

strng='ssh ic4lx86 -T '
strng+=$'\'cd /home/virtuser/EXTRACT-FC && '
strng+=$'/home/virtuser/EXTRACT-FC/install\''
printf '\n>>>%s<<<\n\n' "$strng"
eval $strng


>>>ssh ic4lx86 -T 'cd /home/virtuser/EXTRACT-FC && /home/virtuser/EXTRACT-FC/install'<<<

/usr/bin/sudo rpm -Uvh /home/virtuser/EXTRACT-FC/images/rhel/*.rpm
Verifying...                          ########################################
Preparing...                          ########################################
Updating / installing...
cobol.rte.1.2.0-1.2.0.7-250707        ########################################
cobol.cmp.1.2.0-1.2.0.7-250707        ########################################
cobol.dbg.1.2.0-1.2.0.7-250707        ########################################
Cleaning up / removing...
cobol.cmp.1.2.0-1.2.0.5-241115        ########################################
cobol.rte.1.2.0-1.2.0.5-241115        ########################################
cobol.dbg.1.2.0-1.2.0.5-241115        ########################################
Installation and configuration successful


In [16]:
%%bash
# Validae that version 1.2.0.6, or above, is in place for the PostgreSQL support.

strng='ssh ic4lx86 -T '
strng+=$'\'sudo  dnf repoquery --userinstalled | grep --ignore-case cobol | rev | sort -r | rev | nl\''
printf '\n>>>%s<<<\n\n' "$strng"
eval $strng


>>>ssh ic4lx86 -T 'sudo  dnf repoquery --userinstalled | grep --ignore-case cobol | rev | sort -r | rev | nl'<<<

     1	cobol.cmp.1.2.0-0:1.2.0.7-250707.x86_64
     2	cobol.dbg.1.2.0-0:1.2.0.7-250707.x86_64
     3	cobol.rte.1.2.0-0:1.2.0.7-250707.x86_64
     4	cobol.cmp.license-eval.1.2.0-0:1.2.0.5-241115.x86_64


###  03.03 cob2.cfg modified for `/usr/pgsql-17/lib/`
`cob2` is a binary file that sets the options for compilation and linkage of COBOL program.  Its has variants for different databases systems.  The variant for PostgreSQL is `cob2_pgsql`.  `cob2` finds libraries and configuration data in the the files `cob2.cfg`.  See ["Tailoring your compilation"](https://www.ibm.com/docs/en/cobol-linux-x86/1.2.0?topic=configuration-tailoring-your-compilation) in the IBM Documentation.

In [42]:
%%bash
# Inform cob2_pgsql we are using /usr/pgsql-17/lib.

strng='ssh ic4lx86 -T '
strng+=$'\'sed -n \'s/pgsql-15/pgsql-17/p\' /opt/ibm/cobol/1.2.0/etc/cob2.cfg\''
printf '\n>>>%s<<<\n\n' "$strng"
eval $strng


>>>ssh ic4lx86 -T 'sed -n 's/pgsql-15/pgsql-17/p' /opt/ibm/cobol/1.2.0/etc/cob2.cfg'<<<

	coprocessor    = /usr/pgsql-17/lib
	coprocessor_64 = /usr/pgsql-17/lib
	runlib2        = -lpgsql_32r -L/usr/pgsql-17/lib -lecpg
	runlib2_64     = -lpgsql_64r -L/usr/pgsql-17/lib -lecpg


### 03.04  Tidy up. Remove directories for COBOL install and Fix Central maintenance.

In [24]:
%%bash
# Remove directories and their contents that are no longer needed.

strng='ssh ic4lx86 -T '
strng+=$'\'rm --recursive /home/virtuser/EXTRACT-COBOL && '
strng+=$'rm --recursive /home/virtuser/EXTRACT-FC\''
printf '\n>>>%s<<<\n\n' "$strng"
eval $strng


>>>ssh ic4lx86 -T 'rm --recursive /home/virtuser/EXTRACT-COBOL && rm --recursive /home/virtuser/EXTRACT-FC'<<<



## 04  Compile and run the example program: IBM COBOL accesses PostgreSQL.

In [51]:
%%bash
# Invoke a built for purpose script.

../SCRIPTS/compile-run.sh cbl2pgsql_xmp01


***  2025-08-26T17:35:56:-0500 --  001  sftp cbl2pgsql_xmp01 to the machine with IBM COBOL.  ***
------------------------------------------------------------------------------------------------
>>>sftp ic4lx86:/home/virtuser/COBOL <<< "put ../COBOL/cbl2pgsql_xmp01.cobol"<<<



Connected to ic4lx86.


Changing to: /home/virtuser/COBOL
sftp> put ../COBOL/cbl2pgsql_xmp01.cobol
Uploading ../COBOL/cbl2pgsql_xmp01.cobol to /home/virtuser/COBOL/cbl2pgsql_xmp01.cobol

***  2025-08-26T17:35:57:-0500 --  002  Compile and link-edit to PGSQL using ecpg.  ***
---------------------------------------------------------------------------------------
>>>ssh -T ic4lx86 cob2_pgsql /home/virtuser/COBOL/cbl2pgsql_xmp01.cobol -o cbl2pgsql_xmp01<<<



IBM COBOL for Linux 1.2.0 compile started


End of compilation 1,  program HELLO_POSTGRES,  no statements flagged.

***  2025-08-26T17:35:58:-0500 --  003  Execute the COBOL program. Displays results from PGSQL.  ***
----------------------------------------------------------------------------------------------------
>>>ssh -T ic4lx86 'source /home/virtuser/pgsql_env_vars && /home/virtuser/cbl2pgsql_xmp01'<<<

   emp_names     
---------------- 
 Janet Smith     
 Lee Reynolds    
 Samuel Cole     
 Samantha Bush   
 Betty Diaz      
 Kathleen Roush  
+++ Disconnected from PostgreSQL database. +++

***  2025-08-26T17:36:00:-0500 --  004  Validate the COBOL PGSQL results with psql expression equivalent to the COBOL program.  ***
-----------------------------------------------------------------------------------------------------------------------------------
>>>psql --dbname AD22 --command "SELECT CONCAT(first_name, ' ', last_name) AS emp_names FROM teachers;"<<<

   emp_names    
----------------
 Janet Smith
 Lee Reynolds
 Samue